# Pronóstico automatizado de productos RDS (01.01.2024-15.06.2025) vendidos por seller Repuestosdelsol

## Modelo XGBoost

In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import os
from pipeline.procesamiento_bases import procesar_por_par_producto_marca

path_excel = "/Users/martincarrasco/Desktop/Martín_Carrasco/Reportes/Interanuales/Cuentas RDS/Ventas Consolidado RDS 2024-2025 (hasta 15-06-2025).xlsx"
fecha_limite = "2025-06-01"
seller = 'REPUESTOS DEL SOL'

In [ ]:
df = procesar_por_par_producto_marca(path_excel, seller, fecha_limite)
df

/Users/martincarrasco/Desktop/Martín_Carrasco/pipeline/pipeline/procesamiento_bases.py:10: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  fechas_completas = pd.date_range(start=df['Fecha_Compra'].min(),


,Producto,Marca,Fecha_Compra,Unidades_Vendidas
0,Abrazadera Barra Estab Para Hyundai Tucs,HYUNDAI,2024-01-31,0
1,Abrazadera Barra Estab Para Hyundai Tucs,HYUNDAI,2024-02-29,0
2,Abrazadera Barra Estab Para Hyundai Tucs,HYUNDAI,2024-03-31,0
3,Abrazadera Barra Estab Para Hyundai Tucs,HYUNDAI,2024-04-30,0
4,Abrazadera Barra Estab Para Hyundai Tucs,HYUNDAI,2024-05-31,0
...,...,...,...,...
317929,pticos Tuning Para Nissan V16 2003 2011,DIFORZA,2025-01-31,1
317930,pticos Tuning Para Nissan V16 2003 2011,DIFORZA,2025-02-28,0
317931,pticos Tuning Para Nissan V16 2003 2011,DIFORZA,2025-03-31,0
317932,pticos Tuning Para Nissan V16 2003 2011,DIFORZA,2025-04-30,0


In [4]:
def crear_features(df_input):
    df = df_input.copy()
    df["mes"] = df["Fecha_Compra"].dt.month
    df["año"] = df["Fecha_Compra"].dt.year
    df["lag_1"] = df.groupby(["Producto", "Marca"])["Unidades_Vendidas"].shift(1)
    df["lag_2"] = df.groupby(["Producto", "Marca"])["Unidades_Vendidas"].shift(2)
    df["rolling_3"] = df.groupby(["Producto", "Marca"])["Unidades_Vendidas"].shift(1).rolling(3).mean()
    return df

In [5]:
pares_producto_marca = df[["Producto", 'Marca']].drop_duplicates()
pares_producto_marca

,Producto,Marca
0,Abrazadera Barra Estab Para Hyundai Tucs,HYUNDAI
17,Absorbedor Impacto Original Hyundai Vern,HYUNDAI
34,Absorbedor Impacto Para Original Hyundai,HYUNDAI
51,Absorbedor Impacto Para Renault Captur 2,PULO
68,Acople Conector Tubo Bomba Agua Chevrole,ALTERNATIVO
...,...,...
317849,ptico Derecho Para Geely Ck 2009 2014,GENÉRICO
317866,ptico Izquierdo Para Geely Ck 2009 2014,GENÉRICO
317883,ptico Izquierdo Para Hyundai Accent 200,GENÉRICO
317900,ptico Para Kia Morning 2011 2016 Par,GENÉRICO


In [6]:
resultados_xgb = []

In [8]:
os.makedirs(f'gráficos_pronostico_xgboost_ene25_may25_seller_{seller}', exist_ok = True)

In [ ]:
for idx, row in pares_producto_marca.iterrows():
    producto = row["Producto"]
    marca = row["Marca"]

    df_filtro = df_filtro[(df_filtro['Producto'] == producto) & (df_filtro['Marca'] == marca)]
    df_filtro = crear_features(df_filtro).dropna()

    df_train = df_filtro[df_filtro['Fecha_Compra'] < '2025-01-01']
    df_test = df_filtro[(df_filtro['Fecha_Compra'] >= '2025-01-01') & (df_filtro['Fecha_Compra'] <= '2025-05-31')]

    if (
        len(df_train) < 12 or
        len(df_test) < 5 or
        df_train['Unidades_Vendidas'].sum() = 0 or
        df_test['Unidades_Vendidas'].sum() = 0
    ):
        continue

    try:
        features = ['mes', 'año', 'lag_1', 'lag_2', 'rolling_3']

        X_train = df_train[features]
        y_train = df_train['Unidades_Vendidas']
        X_test = df_test[features]
        y_test = df_test['Unidades_Vendidas']

        model = XGBRegressor(n_estimators = 100, learning_rate = 0.1, random_state = 42)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)
        mae = mean_absolute_error(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))

        resultados_xgb.append({
            'Producto': producto,
            'Marca': marca,
            'MAE': mae,
            'RMSE': rmse
        })

        fechas_test = df_test['Fecha_Compra'].values
        plt.figure(figsize=(10, 5))
        plt.plot(df_filtro["Fecha_Compra"], df_filtro["Unidades_Vendidas"], label="Datos Reales", marker="o")
        plt.plot(fechas_test, y_pred, label="Pronóstico XGBoost", marker="x", linestyle="--", color="green")
        plt.title(f'{producto} - {marca} | MAE: {mae:.1f} | RMSE: {rmse:.1f}')
        plt.xlabel('Fecha')
        plt.ylabel('Unidades Vendidas')
        plt.xticks(rotation=45)
        plt.grid(True)
        plt.legend()
        plt.tight_layout()

        nombre_archivo = f"{producto}_{marca}".replace("/", "_").replace("\\", "_")
        plt.savefig(f"graficos_pronostico_xgboost_ene25_may25_seller_{seller}/{nombre_archivo}")
        plt.close

    except Exception as e:
        print(f'Error con {producto} - {marca}: {e}')
        continue

df_resultados_xgb = pd.DataFrame(resultados_xgb)
